In [34]:
from plotly import tools
import plotly.offline as py
import plotly.graph_objects as go
def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

# SSD
cnt_srs = data.groupby('TypeName')['Price_euros'].agg(['max', 'mean'])
cnt_srs.columns = [ "Maximum price", "mean"]
cnt_srs = cnt_srs.sort_values(by="Maximum price", ascending=False)

trace3 = horizontal_bar_chart(cnt_srs["Maximum price"].head(10), 'rgba(50, 100, 96, 0.6)')
trace4 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(90, 10, 96, 0.8)')


# OS
cnt_srs = data.groupby('OpSys')['Price_euros'].agg(['max', 'mean'])
cnt_srs.columns = [ "Maximum price", "mean"]
cnt_srs = cnt_srs.sort_values(by="Maximum price", ascending=False)

trace1 = horizontal_bar_chart(cnt_srs["Maximum price"].head(10), 'rgba(90, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')


# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.04, 
                          subplot_titles=["Maximum price by OS", "Mean price by OS","Maximum price by type name","Mean price by type name"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)


fig['layout'].update(height=1000, width=1000, paper_bgcolor='rgb(233,233,233)', title="Prices by OS and typename")
py.iplot(fig, filename='Prices')

In [8]:
import pandas as pd
import numpy as np
import re
import math 

data = pd.read_csv("./laptops.csv", encoding = "ISO-8859-1")


def getNumber(string):
    number = 4
    try:
        number = re.findall("\d+", string)[0]
    except TypeError:
        number= 4
    return number

def countPPI(row):
    screen = row["ScreenResolution"]
    diagonal = row["Inches"]
    ppi = 0
    try:
        w = float(re.findall("\d+", screen)[-2])
        h = float(re.findall("\d+", screen)[-1])
        ppi = math.sqrt(w*w + h*h) / diagonal
    except TypeError:
        ppi = 100
    return ppi

def isTouchScreen(screen):
    screen = str(screen)
    if screen.find("Touchscreen") > 0:
        return 1
    return 0

def getFlotNumber(string):
    number = 1
    try:
        number = re.findall("\d+\.\d+", string)[0]
    except:
        number= 1
    
    return number

# 0 - mobile 1- middle 2 - performance
def defineVideocardClass(cardName):
    cardName = str(cardName)
    
    performanceClass = 0
    if "GTX" in cardName or "Quadro" in cardName:
        performanceClass = 2
    if "AMD Radeon" in cardName:
        performanceClass = 1
    return performanceClass

def countMemorySSD(memory):
    memory = str(memory)
    memory = memory.replace("TB", "000GB")
    memory = memory.replace("Flash Storage", "SSD")
    ssdCapacity = 0
    
    if memory == 'nan':
        ssdCapacity = 128
    ssds = re.findall("\d+GB\ SSD", memory)
    
    for ssd in ssds: 
        ssdCapacity = ssdCapacity + int(getNumber(ssd))
    
    return ssdCapacity

def countMemoryHDD(memory):
    memory = str(memory)
    memory = memory.replace("TB", "000GB")
    memory = memory.replace("Flash Storage", "SSD")
    hddCapacity = 0
    hdds = re.findall("\d+GB\ HDD", memory)
    
    for hdd in hdds: 
        hddCapacity = hddCapacity + int(getNumber(hdd))
    
    return hddCapacity

data["Ram"] = data["Ram"].apply(getNumber)
data["isTouchScreen"] = data["ScreenResolution"].apply(isTouchScreen)
data["PPI"] = data.apply(countPPI, axis=1)
data["Weight"] = data["Weight"].apply(getFlotNumber)
data["ProcessorFrequency"] = data["Cpu"].apply(getFlotNumber)
data["GPU_performance_rate"] = data["Gpu"].apply(defineVideocardClass)
data["SSD"] = data["Memory"].apply(countMemorySSD)
data["HDD"] = data["Memory"].apply(countMemoryHDD)



data = data.drop(["0"],axis=1)
data = data.drop(["Product"],axis=1)
data = data.drop(["ScreenResolution"],axis=1)
data = data.drop(["Cpu"],axis=1)
data = data.drop(["Gpu"],axis=1)
data = data.drop(["Memory"],axis=1)

data

,Company,TypeName,Inches,Ram,OpSys,Weight,Price_euros,isTouchScreen,PPI,ProcessorFrequency,GPU_performance_rate,SSD,HDD
0,Apple,Ultrabook,13.3,8,macOS,1.37,1339.69,0,226.983005,2.3,0,128,0
1,Apple,Ultrabook,13.3,8,macOS,1.34,898.94,0,127.677940,1.8,0,128,0
2,HP,Notebook,15.6,8,No OS,1.86,575.00,0,141.211998,2.5,0,256,0
3,Apple,Ultrabook,15.4,16,macOS,1.83,2537.45,0,220.534624,2.7,1,512,0
4,Apple,Ultrabook,13.3,8,macOS,1.37,1803.60,0,226.983005,3.1,0,256,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,14.0,4,Windows 10,1.8,638.00,1,157.350512,2.5,0,128,0
1299,Lenovo,2 in 1 Convertible,13.3,16,Windows 10,1.3,1499.00,1,276.053530,2.5,0,512,0
1300,Lenovo,Notebook,14.0,2,Windows 10,1.5,229.00,0,111.935204,1.6,0,64,0
1301,HP,Notebook,15.6,6,Windows 10,2.19,764.00,0,100.454670,2.5,1,0,1000


In [36]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.offline as py
import plotly.figure_factory as ff
from plotly.offline import iplot

import numpy as np
import pandas as pd
import scipy
# Среднее и стандартное отклонение 
print('mean=%.3f stdv=%.3f' % (np.mean(data.Price_euros), np.std(data.Price_euros)))
trace = go.Histogram(
    x=data.Price_euros
    
)

py.iplot([trace], filename='normality-histogram')

mean=1124.317 stdv=699.923
